### Imports, pengaturan slot waktu, parameter global

In [1]:
import random
import math
import copy
from collections import defaultdict
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
import pyswarms as ps

random.seed(42)
np.random.seed(42)

# ---------- Scheduling parameters ----------
DAYS = ["Mon","Tue","Wed","Thu","Fri"]  # Sat/Sun libur
DAY_NAME_MAP = {"Mon":"Senin","Tue":"Selasa","Wed":"Rabu","Thu":"Kamis","Fri":"Jumat"}
START_TIME = datetime.strptime("07:00","%H:%M")
END_TIME   = datetime.strptime("18:00","%H:%M")
SLOT_MINUTES = 75  # setiap mata kuliah 75 menit (1 jam 15 menit)

# Hard constraint penalty (very large)
P_HARD = 10000
# Soft penalties multipliers
P_PRIORITY = 5
P_FLOOR_MOVE = 1
P_PREFERENCE = 20

# For testing / speed, set small population / iterations
GA_POP = 60
GA_GEN = 60
PSO_ITERS = 60

### Dummy data (courses, rooms, labs)

In [2]:
# ---------- Dummy courses (cleaned & simplified) ----------
courses_data = [
    {"kode":"CIE61101","nama":"Perkembangan Peserta Didik","prodi":"PTI","priority":1,"room_type":"any","teacher":"39"},
    {"kode":"CIE61102","nama":"Perkembangan Peserta Didik","prodi":"PTI","priority":2,"room_type":"any","teacher":"59"},
    {"kode":"CIE61103","nama":"Filsafat Pendidikan","prodi":"PTI","priority":1,"room_type":"any","teacher":"103"},
    {"kode":"CIE61104","nama":"Komunikasi & Teknologi","prodi":"PTI","priority":1,"room_type":"any","teacher":"39"},
    {"kode":"CIE61108","nama":"Penilaian Hasil Belajar","prodi":"PTI","priority":1,"room_type":"any","teacher":"100"},
    {"kode":"CIE61109","nama":"Pembelajaran Mikro","prodi":"PTI","priority":1,"room_type":"any","teacher":"39"},
    {"kode":"CIE61110","nama":"Perancangan Pembelajaran","prodi":"PTI","priority":1,"room_type":"lab","teacher":"39"},
    {"kode":"CIE61111","nama":"PLP 1","prodi":"PTI","priority":1,"room_type":"any","teacher":"103"},
    {"kode":"REL110","nama":"Agama Kristen","prodi":"PTI","priority":1,"room_type":"any","teacher":"200"}, # contoh agama selain islam
]

df_courses = pd.DataFrame(courses_data)
df_courses.index.name = "course_id"
df_courses = df_courses.reset_index()
N_COURSES = len(df_courses)
df_courses

,course_id,kode,nama,prodi,priority,room_type,teacher
0,0,CIE61101,Perkembangan Peserta Didik,PTI,1,any,39
1,1,CIE61102,Perkembangan Peserta Didik,PTI,2,any,59
2,2,CIE61103,Filsafat Pendidikan,PTI,1,any,103
3,3,CIE61104,Komunikasi & Teknologi,PTI,1,any,39
4,4,CIE61108,Penilaian Hasil Belajar,PTI,1,any,100
5,5,CIE61109,Pembelajaran Mikro,PTI,1,any,39
6,6,CIE61110,Perancangan Pembelajaran,PTI,1,lab,39
7,7,CIE61111,PLP 1,PTI,1,any,103
8,8,REL110,Agama Kristen,PTI,1,any,200


In [3]:
# ---------- Rooms dataframe ----------
rooms = [
    {"kode_ruang":"F11.1","lokasi":"Gedung F FILKOM","lantai":11,"keterangan":"R. Kuliah"},
    {"kode_ruang":"F11.3","lokasi":"Gedung F FILKOM","lantai":11,"keterangan":"R. Kuliah"},
    {"kode_ruang":"F2.1","lokasi":"Gedung F FILKOM","lantai":2,"keterangan":"Smart Class"},
    {"kode_ruang":"F2.2","lokasi":"Gedung F FILKOM","lantai":2,"keterangan":"Smart Class"},
    {"kode_ruang":"LabJBI1","lokasi":"Gedung Lab","lantai":1,"keterangan":"Lab","is_lab":True},
    {"kode_ruang":"LabJBI2","lokasi":"Gedung Lab","lantai":2,"keterangan":"Lab","is_lab":True}
]
df_rooms = pd.DataFrame(rooms)
df_rooms["is_lab"] = df_rooms.get("is_lab", False).fillna(False)
R = len(df_rooms)
df_rooms


C:\Users\ASUS\AppData\Local\Temp\ipykernel_8988\2195928246.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_rooms["is_lab"] = df_rooms.get("is_lab", False).fillna(False)


,kode_ruang,lokasi,lantai,keterangan,is_lab
0,F11.1,Gedung F FILKOM,11,R. Kuliah,False
1,F11.3,Gedung F FILKOM,11,R. Kuliah,False
2,F2.1,Gedung F FILKOM,2,Smart Class,False
3,F2.2,Gedung F FILKOM,2,Smart Class,False
4,LabJBI1,Gedung Lab,1,Lab,True
5,LabJBI2,Gedung Lab,2,Lab,True


### Build slot indices sebagai interval (day, start_str, end_str), melewati aturan istirahat

In [4]:
slot_indices = []  # list of tuples (day, start_str, end_str)
for d in DAYS:
    start = START_TIME
    while start + timedelta(minutes=SLOT_MINUTES) <= END_TIME:
        end = start + timedelta(minutes=SLOT_MINUTES)
        # skip if this start time falls into break windows
        # Senin-Kamis skip start times that are inside [12:10,12:30)
        if d in ["Mon","Tue","Wed","Thu"]:
            # if start >= 12:10 AND start < 12:30 -> skip
            if datetime.strptime("12:10","%H:%M") <= start < datetime.strptime("12:30","%H:%M"):
                start += timedelta(minutes=SLOT_MINUTES)
                continue
        # Friday skip [11:15,12:30)
        if d == "Fri":
            if datetime.strptime("11:15","%H:%M") <= start < datetime.strptime("12:30","%H:%M"):
                start += timedelta(minutes=SLOT_MINUTES)
                continue
        # otherwise add
        slot_indices.append((d, start.strftime("%H:%M"), end.strftime("%H:%M")))
        start += timedelta(minutes=SLOT_MINUTES)

NUM_SLOTS = len(slot_indices)
SLOT_INDEX_MAP = { (day, start): idx for idx, (day, start, end) in enumerate(slot_indices) }
TOTAL_POS = NUM_SLOTS * R

print(f"Total slots (day x intervals): {NUM_SLOTS}")
# preview first 8 slots
slot_indices[:8]

# For convenience, build slot_allowed mask (True for all generated slots)
slot_allowed = np.ones(NUM_SLOTS, dtype=bool)

# For demonstration: assign a couple of preferred slots for teachers (soft)
# Represent preference as (day_str, start_hour_int) e.g. ("Mon",9) -> 09:00 start slot if available
teacher_preferences = {
    "39": ("Mon",9),   # teacher 39 prefers Monday starting 09:00
    "103":("Tue",10)
}
def pref_to_slot(pref):
    d,h = pref
    target = f"{h:02d}:00"
    # find slot with same day and start HH:MM starting with target hour
    for idx,(day,s,e) in enumerate(slot_indices):
        if day==d and s.startswith(f"{h:02d}:"):
            return idx
    return None

teacher_pref_idx = {}
for tid,p in teacher_preferences.items():
    s = pref_to_slot(p)
    if s is not None:
        teacher_pref_idx[tid]=[s]
teacher_pref_idx

Total slots (day x intervals): 39


{'39': [2], '103': [11]}

### Encoding/decoding & fitness function (hard constraints + soft)

In [5]:
# Encoding scheme: gene per course -> integer in [0, NUM_SLOTS * R - 1]

def encode(slot_idx, room_idx):
    return slot_idx * R + room_idx

def decode(gene):
    slot_idx = int(gene) // R
    room_idx = int(gene) % R
    return slot_idx, room_idx

# Helper: check if a room is lab
def room_is_lab(room_idx):
    return bool(df_rooms.loc[room_idx, "is_lab"])

# Convert slot_idx back to (day,start,end)
def slot_to_dayhour(slot_idx):
    return slot_indices[slot_idx]  # (day, start_str, end_str)

# Fitness function: lower better (we'll minimize cost)
def fitness_of_solution(genome):
    # genome: list of length N_COURSES with ints
    cost = 0
    # maps for constraints
    room_time_usage = defaultdict(list)  # slot_idx -> list of (room_idx, course_i)
    teacher_time = defaultdict(list)     # slot_idx -> list of (teacher, course_i)
    course_assign = {}
    # decode and validate slot allowed
    for i,g in enumerate(genome):
        slot_idx, room_idx = decode(g)
        # invalid slot or room index -> heavy penalty
        if slot_idx < 0 or slot_idx >= NUM_SLOTS or room_idx<0 or room_idx>=R:
            cost += P_HARD
            continue
        # blocked slot (shouldn't happen because generation skipped breaks) but keep safety
        if not slot_allowed[slot_idx]:
            cost += P_HARD
        # lab requirement
        need_lab = (df_courses.loc[i,"room_type"] == "lab")
        if need_lab and not room_is_lab(room_idx):
            cost += P_HARD
        # agama selain islam pref: prefer Friday (soft)
        if "Agama" in df_courses.loc[i,"nama"]:
            day,_,_ = slot_to_dayhour(slot_idx)
            if day != "Fri":
                cost += P_PRIORITY*2

        # collect usages
        room_time_usage[slot_idx].append((room_idx,i))
        teacher_time[slot_idx].append((df_courses.loc[i,"teacher"], i))
        course_assign[i] = (slot_idx, room_idx)

    # Hard constraint 1: no two courses same room same slot
    for slot_idx, lst in room_time_usage.items():
        used_rooms = {}
        for room_idx,i in lst:
            if room_idx in used_rooms:
                # conflict: two courses in same room same time
                cost += P_HARD
            else:
                used_rooms[room_idx] = i

    # Hard constraint 2,3: teacher cannot be in two places same slot
    for slot_idx, lst in teacher_time.items():
        teachers_seen = {}
        for teacher,i in lst:
            if not teacher or pd.isna(teacher): continue
            if teacher in teachers_seen:
                # teacher teach >1 course same slot
                cost += P_HARD
            else:
                teachers_seen[teacher]=i

    # Soft constraint: priority -> if in same slot multiple courses prefer lower priority value
    slot_courses = defaultdict(list)
    for i,(slot_idx,room_idx) in course_assign.items():
        slot_courses[slot_idx].append(i)
    for slot_idx, lst in slot_courses.items():
        if len(lst)<=1: continue
        prior_sum = sum(df_courses.loc[i,"priority"] for i in lst)
        cost += P_PRIORITY * (prior_sum - min(df_courses.loc[i,"priority"] for i in lst))

    # Soft constraint: minimize teacher movement by floors (sum abs diffs)
    teacher_rooms = defaultdict(list)
    for i,(slot_idx,room_idx) in course_assign.items():
        t = df_courses.loc[i,"teacher"]
        if pd.isna(t): continue
        teacher_rooms[t].append(room_idx)
    for t,rooms_list in teacher_rooms.items():
        if len(rooms_list)<=1: continue
        floors = [df_rooms.loc[r,"lantai"] for r in rooms_list]
        for a,b in zip(floors, floors[1:]):
            cost += P_FLOOR_MOVE * abs(a-b)

    # Soft: teacher time preferences
    for i,(slot_idx,room_idx) in course_assign.items():
        t = df_courses.loc[i,"teacher"]
        if t in teacher_pref_idx:
            prefs = teacher_pref_idx[t]
            if slot_idx not in prefs:
                cost += P_PREFERENCE

    return cost

# Test fitness with a random genome
sample_genome = [random.randint(0, TOTAL_POS-1) for _ in range(N_COURSES)]
print("Sample cost:", fitness_of_solution(sample_genome))

Sample cost: 10153


### Utility to pretty-print solution

In [6]:
def pretty_print_solution(genome):
    rows = []
    for i,g in enumerate(genome):
        slot_idx, room_idx = decode(g)
        day, start, end = slot_to_dayhour(slot_idx)
        rows.append({
            "course_id": i,
            "kode": df_courses.loc[i,"kode"],
            "nama": df_courses.loc[i,"nama"],
            "teacher": df_courses.loc[i,"teacher"],
            "day": DAY_NAME_MAP.get(day, day),
            "time": f"{start} - {end}",
            "room": df_rooms.loc[room_idx,"kode_ruang"],
            "room_floor": df_rooms.loc[room_idx,"lantai"],
            "priority": df_courses.loc[i,"priority"]
        })
    return pd.DataFrame(rows).sort_values(["day","time","room"])

# show a sample
pretty_print_solution(sample_genome)

,course_id,kode,nama,teacher,day,time,room,room_floor,priority
0,0,CIE61101,Perkembangan Peserta Didik,39,Kamis,10:45 - 12:00,F11.3,11,1
8,8,REL110,Agama Kristen,200,Kamis,15:45 - 17:00,F2.1,2,1
3,3,CIE61104,Komunikasi & Teknologi,39,Kamis,15:45 - 17:00,F2.2,2,1
6,6,CIE61110,Perancangan Pembelajaran,39,Selasa,08:15 - 09:30,F2.2,2,1
5,5,CIE61109,Pembelajaran Mikro,39,Selasa,09:30 - 10:45,F2.1,2,1
4,4,CIE61108,Penilaian Hasil Belajar,100,Selasa,10:45 - 12:00,LabJBI1,1,1
2,2,CIE61103,Filsafat Pendidikan,103,Senin,08:15 - 09:30,F11.1,11,1
1,1,CIE61102,Perkembangan Peserta Didik,59,Senin,12:00 - 13:15,LabJBI1,1,2
7,7,CIE61111,PLP 1,103,Senin,13:15 - 14:30,LabJBI2,2,1


### Genetic Algorithm (DEAP)

In [7]:
# GA setup: minimize fitness
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# gene init: uniform choose total positions
toolbox.register("attr_gene", random.randint, 0, TOTAL_POS-1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_gene, n=N_COURSES)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_individual(individual):
    return (fitness_of_solution(individual),)

toolbox.register("evaluate", eval_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=TOTAL_POS-1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def run_ga(pop_size=GA_POP, gens=GA_GEN):
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.2, ngen=gens, stats=stats, halloffame=hof, verbose=False)
    return hof[0], log

best_ga, log = run_ga()
print("GA best cost:", fitness_of_solution(best_ga))
pretty_print_solution(best_ga)

GA best cost: 81


,course_id,kode,nama,teacher,day,time,room,room_floor,priority
2,2,CIE61103,Filsafat Pendidikan,103,Jumat,13:15 - 14:30,F2.2,2,1
8,8,REL110,Agama Kristen,200,Jumat,15:45 - 17:00,LabJBI2,2,1
4,4,CIE61108,Penilaian Hasil Belajar,100,Kamis,12:00 - 13:15,F11.1,11,1
0,0,CIE61101,Perkembangan Peserta Didik,39,Rabu,09:30 - 10:45,LabJBI2,2,1
5,5,CIE61109,Pembelajaran Mikro,39,Rabu,12:00 - 13:15,LabJBI1,1,1
7,7,CIE61111,PLP 1,103,Selasa,10:45 - 12:00,F2.2,2,1
3,3,CIE61104,Komunikasi & Teknologi,39,Selasa,13:15 - 14:30,F2.1,2,1
1,1,CIE61102,Perkembangan Peserta Didik,59,Senin,08:15 - 09:30,F2.1,2,2
6,6,CIE61110,Perancangan Pembelajaran,39,Senin,09:30 - 10:45,LabJBI1,1,1


### Simulated Annealing (simple custom)

In [8]:
def neighbor(genome):
    g = genome.copy()
    i = random.randrange(len(g))
    g[i] = random.randint(0, TOTAL_POS-1)
    return g

def simulated_annealing(init_genome=None, iters=2000, start_temp=1000, cooling=0.995):
    if init_genome is None:
        current = [random.randint(0,TOTAL_POS-1) for _ in range(N_COURSES)]
    else:
        current = init_genome.copy()
    best = current.copy()
    best_cost = fitness_of_solution(best)
    temp = start_temp
    for k in range(iters):
        cand = neighbor(current)
        cand_cost = fitness_of_solution(cand)
        cur_cost = fitness_of_solution(current)
        if cand_cost < cur_cost or random.random() < math.exp((cur_cost-cand_cost)/max(temp,1e-9)):
            current = cand
        if cand_cost < best_cost:
            best = cand
            best_cost = cand_cost
        temp *= cooling
    return best, best_cost

best_sa, cost_sa = simulated_annealing(iters=2000)
print("SA best cost:", cost_sa)
pretty_print_solution(best_sa)

SA best cost: 80


,course_id,kode,nama,teacher,day,time,room,room_floor,priority
6,6,CIE61110,Perancangan Pembelajaran,39,Jumat,07:00 - 08:15,LabJBI2,2,1
8,8,REL110,Agama Kristen,200,Jumat,13:15 - 14:30,F2.1,2,1
7,7,CIE61111,PLP 1,103,Kamis,13:15 - 14:30,F2.2,2,1
4,4,CIE61108,Penilaian Hasil Belajar,100,Rabu,07:00 - 08:15,F2.1,2,1
5,5,CIE61109,Pembelajaran Mikro,39,Rabu,12:00 - 13:15,LabJBI2,2,1
2,2,CIE61103,Filsafat Pendidikan,103,Selasa,10:45 - 12:00,LabJBI2,2,1
0,0,CIE61101,Perkembangan Peserta Didik,39,Selasa,15:45 - 17:00,F2.1,2,1
3,3,CIE61104,Komunikasi & Teknologi,39,Senin,09:30 - 10:45,F2.1,2,1
1,1,CIE61102,Perkembangan Peserta Didik,59,Senin,10:45 - 12:00,F2.2,2,2


### PSO (pyswarms) simple discrete mapping

In [9]:
# We'll use continuous PSO with dimensions = N_COURSES,
# each particle coordinate in [0, TOTAL_POS). We'll round to int and decode.

def pso_fitness(x):
    # x shape (n_particles, N_COURSES)
    n = x.shape[0]
    out = np.zeros(n)
    for i in range(n):
        genome = [int(max(0, min(TOTAL_POS-1, round(val)))) for val in x[i]]
        out[i] = fitness_of_solution(genome)
    return out

# configure optimizer
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=30, dimensions=N_COURSES, options=options, bounds=(np.zeros(N_COURSES), np.ones(N_COURSES)*(TOTAL_POS-1)))
best_cost, best_pos = optimizer.optimize(pso_fitness, iters=PSO_ITERS, verbose=False)
best_genome_pso = [int(round(x)) for x in best_pos]
print("PSO best cost:", fitness_of_solution(best_genome_pso))
pretty_print_solution(best_genome_pso)

PSO best cost: 110


,course_id,kode,nama,teacher,day,time,room,room_floor,priority
0,0,CIE61101,Perkembangan Peserta Didik,39,Jumat,07:00 - 08:15,LabJBI1,1,1
1,1,CIE61102,Perkembangan Peserta Didik,59,Kamis,13:15 - 14:30,F2.2,2,2
6,6,CIE61110,Perancangan Pembelajaran,39,Rabu,08:15 - 09:30,LabJBI1,1,1
4,4,CIE61108,Penilaian Hasil Belajar,100,Rabu,09:30 - 10:45,F2.2,2,1
5,5,CIE61109,Pembelajaran Mikro,39,Selasa,07:00 - 08:15,LabJBI1,1,1
2,2,CIE61103,Filsafat Pendidikan,103,Selasa,10:45 - 12:00,F11.3,11,1
7,7,CIE61111,PLP 1,103,Selasa,13:15 - 14:30,F11.1,11,1
8,8,REL110,Agama Kristen,200,Selasa,14:30 - 15:45,LabJBI2,2,1
3,3,CIE61104,Komunikasi & Teknologi,39,Selasa,15:45 - 17:00,LabJBI1,1,1


### Ant Colony Optimization

In [10]:
# Simple ACO: pheromone matrix over positions for each course (course-specific choices).
def aco(num_ants=40, iterations=80, alpha=1.0, beta=1.0, rho=0.1):
    # pheromone shape (N_COURSES, TOTAL_POS)
    pher = np.ones((N_COURSES, TOTAL_POS))
    # heuristic: inverse of local penalty (1/(1+cost_if_assign_here))
    heuristic = np.zeros_like(pher)
    for i in range(N_COURSES):
        for pos in range(TOTAL_POS):
            genome = [random.randint(0, TOTAL_POS-1) for _ in range(N_COURSES)]
            genome[i] = pos
            heuristic[i,pos] = 1.0 / (1.0 + fitness_of_solution(genome))
    best_sol = None
    best_cost = float("inf")
    for it in range(iterations):
        ants_solutions = []
        ants_costs = []
        for a in range(num_ants):
            sol = []
            for i in range(N_COURSES):
                probs = (pher[i]**alpha) * (heuristic[i]**beta)
                probs = probs / probs.sum()
                choice = np.random.choice(range(TOTAL_POS), p=probs)
                sol.append(int(choice))
            cost = fitness_of_solution(sol)
            ants_solutions.append(sol)
            ants_costs.append(cost)
            if cost < best_cost:
                best_cost = cost
                best_sol = sol.copy()
        # pheromone update
        pher *= (1 - rho)
        # deposit based on quality
        for sol,cost in zip(ants_solutions, ants_costs):
            deposit = 1.0 / (1.0 + cost)
            for i,pos in enumerate(sol):
                pher[i,pos] += deposit
    return best_sol, best_cost

best_aco, cost_aco = aco(iterations=80)
print("ACO best cost:", cost_aco)
pretty_print_solution(best_aco)

ACO best cost: 90


,course_id,kode,nama,teacher,day,time,room,room_floor,priority
4,4,CIE61108,Penilaian Hasil Belajar,100,Kamis,15:45 - 17:00,F11.1,11,1
8,8,REL110,Agama Kristen,200,Rabu,08:15 - 09:30,F2.2,2,1
0,0,CIE61101,Perkembangan Peserta Didik,39,Rabu,14:30 - 15:45,F2.2,2,1
1,1,CIE61102,Perkembangan Peserta Didik,59,Selasa,07:00 - 08:15,F11.1,11,2
7,7,CIE61111,PLP 1,103,Selasa,10:45 - 12:00,F11.1,11,1
3,3,CIE61104,Komunikasi & Teknologi,39,Senin,07:00 - 08:15,F2.1,2,1
5,5,CIE61109,Pembelajaran Mikro,39,Senin,09:30 - 10:45,F2.2,2,1
6,6,CIE61110,Perancangan Pembelajaran,39,Senin,10:45 - 12:00,LabJBI2,2,1
2,2,CIE61103,Filsafat Pendidikan,103,Senin,14:30 - 15:45,F11.1,11,1


### Improved GA (elitism + local swap hill-climb) & Improved PSO (adaptive inertia)

In [11]:
# Improved GA: after GA finishes, apply local pairwise swap hill-climb on best
def improved_ga():
    best, _ = run_ga(pop_size=GA_POP, gens=GA_GEN)
    best = list(best)
    improved = best.copy()
    best_cost = fitness_of_solution(improved)
    # local improvement: try swapping room/slot between pairs and keep if better
    for _ in range(200):
        i,j = random.sample(range(N_COURSES),2)
        new = improved.copy()
        new[i], new[j] = new[j], new[i]
        c = fitness_of_solution(new)
        if c < best_cost:
            improved = new
            best_cost = c
    return improved, best_cost

# Improved PSO: adaptive inertia decreasing with iteration (we can simulate by running PSO multiple times with different w)
def improved_pso():
    # run PSO with decreasing inertia via manual loop: run small PSO stages
    global_best = None
    global_best_cost = float("inf")
    for w in [0.9, 0.7, 0.5, 0.3]:
        options = {'c1':0.5,'c2':0.3,'w':w}
        opt = ps.single.GlobalBestPSO(n_particles=30, dimensions=N_COURSES, options=options, bounds=(np.zeros(N_COURSES), np.ones(N_COURSES)*(TOTAL_POS-1)))
        best_cost_stage, best_pos_stage = opt.optimize(pso_fitness, iters=20, verbose=False)
        genome = [int(round(x)) for x in best_pos_stage]
        c = fitness_of_solution(genome)
        if c < global_best_cost:
            global_best_cost = c
            global_best = genome
    # apply small local search
    improved = global_best.copy()
    for _ in range(200):
        i = random.randrange(N_COURSES)
        old = improved[i]
        improved[i] = random.randint(0,TOTAL_POS-1)
        c = fitness_of_solution(improved)
        if c < global_best_cost:
            global_best_cost = c
        else:
            improved[i] = old
    return improved, global_best_cost

best_ig, cost_ig = improved_ga()
print("Improved GA cost:", cost_ig)
pretty_print_solution(best_ig)

best_ipso, cost_ipso = improved_pso()
print("Improved PSO cost:", cost_ipso)
pretty_print_solution(best_ipso)

Improved GA cost: 80
Improved PSO cost: 81


,course_id,kode,nama,teacher,day,time,room,room_floor,priority
8,8,REL110,Agama Kristen,200,Jumat,07:00 - 08:15,F2.1,2,1
3,3,CIE61104,Komunikasi & Teknologi,39,Jumat,08:15 - 09:30,LabJBI2,2,1
2,2,CIE61103,Filsafat Pendidikan,103,Kamis,10:45 - 12:00,LabJBI2,2,1
6,6,CIE61110,Perancangan Pembelajaran,39,Kamis,13:15 - 14:30,LabJBI1,1,1
0,0,CIE61101,Perkembangan Peserta Didik,39,Kamis,15:45 - 17:00,F2.1,2,1
1,1,CIE61102,Perkembangan Peserta Didik,59,Rabu,07:00 - 08:15,LabJBI1,1,2
7,7,CIE61111,PLP 1,103,Selasa,10:45 - 12:00,F2.2,2,1
4,4,CIE61108,Penilaian Hasil Belajar,100,Selasa,12:00 - 13:15,LabJBI1,1,1
5,5,CIE61109,Pembelajaran Mikro,39,Senin,09:30 - 10:45,LabJBI1,1,1


### Ringkasan hasil & perbandingan

In [12]:
results = {
    "GA": fitness_of_solution(best_ga),
    "SA": cost_sa,
    "PSO": fitness_of_solution(best_genome_pso),
    "ACO": cost_aco,
    "Improved_GA": cost_ig,
    "Improved_PSO": cost_ipso
}
pd.Series(results).sort_values()

SA               80
Improved_GA      80
Improved_PSO     81
GA               81
ACO              90
PSO             110
dtype: int64